<a href="https://colab.research.google.com/github/Komalsai234/nl2sql/blob/main/notebooks/nl2sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NL2SQL Research Notebook